In [59]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split as tts 


In [60]:
data=pd.read_csv('Churn_Modelling.csv')

In [61]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [62]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [63]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le=LabelEncoder()
Oh=OneHotEncoder()
data['Gender']=le.fit_transform(data['Gender'])
geo_encoded=Oh.fit_transform(data[['Geography']])


In [64]:
Oh.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [65]:
geo_df=pd.DataFrame(geo_encoded.toarray(),columns=Oh.get_feature_names_out(['Geography']))

In [66]:
data=data.drop('Geography',axis=1)
data=pd.concat([data,geo_df],axis=1)

In [67]:
### SAVING THE FILE 
import pickle
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(le,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(Oh,file)

In [68]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [69]:
### Divide the dataset into independent and dependent 
from sklearn.preprocessing import StandardScaler
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and testing set 
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.2,random_state=42)

## Scale these features
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [70]:
with open('scaler.pkl','wb') as file:
    pickle.dump(sc,file)

## ANN Implementation

In [71]:
import tensorflow as tf 
from tensorflow.keras import models,layers
import datetime

In [72]:
(X_train.shape[1],)

(12,)

In [73]:
model=models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(132,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [74]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 132)            │         1,716 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           133 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,849 (7.22 KB)

 Trainable params: 1,849 (7.22 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()


In [76]:
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000012F42E394E0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [77]:
opt

In [78]:
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [79]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [80]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [81]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8307 - loss: 0.3945 - val_accuracy: 0.8535 - val_loss: 0.3532
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8493 - loss: 0.3581 - val_accuracy: 0.8430 - val_loss: 0.3760
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8560 - loss: 0.3543 - val_accuracy: 0.8590 - val_loss: 0.3474
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8561 - loss: 0.3488 - val_accuracy: 0.8615 - val_loss: 0.3477
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8569 - loss: 0.3451 - val_accuracy: 0.8635 - val_loss: 0.3354
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8591 - loss: 0.3413 - val_accuracy: 0.8630 - val_loss: 0.3385
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8585 - loss: 0.3385 - val_accuracy: 0.8610 - val_loss: 0.3470
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8610 - loss: 0.3419 - val_accu

In [82]:
model.save('model.h5')

In [84]:
%load_ext tensorboard

In [86]:
!tensorboard --logdir logs --port 6006


2025-12-08 21:10:37.463771: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-08 21:10:39.237343: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
E1208 21:10:40.965412 12580 program.py:300] TensorBoard could not bind to port 6006, it was already in use
ERROR: TensorBoard could not bind to port 6006, it was already in use


In [88]:
!taskkill /IM tensorboard.exe /F


SUCCESS: The process "tensorboard.exe" with PID 18352 has been terminated.
